In [ ]:
import os
#from helpers import *

from jlib.cifar_preprocessing import get_cifar_loaders, delete_deletables
from jlib.classifier import Classifier
from jlib.vggnet import VggNet, VggBlock, ConvParams
import torch
from torch import nn
import matplotlib.pyplot as plt
device = 'cuda'

"""
sudo fuser -v -k /usr/lib/wsl/drivers/nvhm.inf_amd64_5c197d2d97068bef/*
"""
    




def train_and_plot(model: Classifier, train, val, title, min_accuracy=0.65):
    print(f"Model: {title}")
    print("-"*50)
    print(f"Num Params: {sum(p.numel() for p in model.parameters())}")
    model.train_model(
        epochs=50,
        train_loader=train,
        val_loader=val,
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam,
        optimizer_args=[],
        optimizer_kwargs={'lr': 1e-4 },
        print_epoch=1,
        header_epoch=10,
        sched_factor=0.1,
        sched_patience=5,
        min_accuracy = 1,
        max_negative_diff_count = 7
    )
    torch.save(model, f'models/{title}.pth')
    fig = model.plot_training(f"Training {title}")
    fig.savefig(f'figures/{title}.png')
    plt.show()
    #plt.savefig(f'figures/{title}.png')

# Num Params original vgg-16 is 3.375870e+08
architecture = {
    'in_chan': 3,
    'in_dim': (32, 32),
    'block_params': [
        VggBlock(
            params=ConvParams(kernel=3,out_chan=256),
            pool_kernel=2,
            pool_stride=2,
            repititions=2
        ),
        VggBlock(
            params=ConvParams(kernel=3,out_chan=512),
            pool_kernel=2,
            pool_stride=1,
            repititions=2
        ),
        VggBlock(
            params=ConvParams(kernel=3,out_chan=512),
            pool_kernel=2,
            pool_stride=1,
            repititions=3
        ),
        VggBlock(
            params=ConvParams(kernel=3,out_chan=256),
            pool_kernel=2,
            pool_stride=1,
            repititions=3
        ),
        VggBlock(
            params=ConvParams(kernel=3,out_chan=128),
            pool_kernel=2,
            pool_stride=1,
            repititions=3
        ),
    ],
    'fc_params': [
        2048,
        2048,
    ],
}
torch.cuda.empty_cache()
Vgg_100_dp = VggNet(
    num_classes=100,
    dropout = 0.5,
    **architecture
).to(device)


cifar_100_train_loader, cifar_100_val_loader = get_cifar_loaders(
    is_cifar10=False,
    train_batch_size=512,
    val_batch_size=2048,
)



#train_and_plot(Vgg_100_dp, cifar_100_train_loader, cifar_100_val_loader, "VggNet CIFAR100 DO")
delete_deletables([Vgg_100_dp])


torch.cuda.empty_cache()
Vgg_100_ndp = VggNet(
    num_classes=100,
    dropout = 0,
    **architecture
).to(device)

train_and_plot(Vgg_100_ndp, cifar_100_train_loader, cifar_100_val_loader, "VggNet CIFAR100 NO DO")
delete_deletables([Vgg_100_ndp])

delete_deletables([cifar_100_train_loader, cifar_100_val_loader])

cifar_10_train_loader, cifar_10_val_loader = get_cifar_loaders(
    is_cifar10=True,
    train_batch_size=512,
    val_batch_size=512,
)

torch.cuda.empty_cache()
Vgg_10_dp = VggNet(
    num_classes=10,
    dropout = 0.5,
    **architecture
).to(device)

train_and_plot(Vgg_10_dp, cifar_10_train_loader, cifar_10_val_loader, "VggNet CIFAR10 DO")
delete_deletables([Vgg_10_dp])

torch.cuda.empty_cache()
Vgg_10_ndp = VggNet(
    num_classes=10,
    dropout = 0,
    **architecture
).to(device)

train_and_plot(Vgg_10_ndp, cifar_10_train_loader, cifar_10_val_loader, "VggNet CIFAR10 NO DO")
del Vgg_10_ndp


